Develope image-like dataset.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
plt.style.use('seaborn-colorblind')
#mpl.rcParams['xtick.labelsize'] = 8
#mpl.rcParams["image.cmap"] = 'gray'
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

import datetime
date = datetime.datetime.now().strftime('%Y%m%d')

%matplotlib inline

In [3]:
path = '/home/users/aslee/WASA_faciesXRF/'
X = np.genfromtxt('{}data/XRF_ML_ravel_X.csv'.format(path), delimiter=',')
y = np.genfromtxt('{}data/XRF_ML_ravel_y.csv'.format(path), delimiter=',')
groups = np.genfromtxt('{}data/XRF_ML_ravel_g.csv'.format(path), delimiter=',')
X.shape
groups.shape

(153357, 204)

In [4]:
data_df = pd.read_csv('data/XRF_ML_cr.csv')
data_df.shape

(153357, 27)

# Read raw data

In [3]:
data_df = pd.read_csv('data/XRF_ML.csv')
data_df.head()

,composite_id,composite_depth_mm,core_id,core_section,section_depth_mm,facies_merge_2,Ba,Br,Ca,Cl,Fe,K,Rb,S,Si,Sr,Ti,Zr
0,N10_00003,3,N10,N10-5,3.0,0,149,0,6327,674,2384,2693,124,57,5522,158,1408,1549
1,N10_00005,5,N10,N10-5,5.0,0,125,8,7200,643,2171,2795,170,40,5153,205,1025,1369
2,N10_00007,7,N10,N10-5,7.0,0,102,11,5992,596,1879,2673,131,7,5208,207,1225,1199
3,N10_00009,9,N10,N10-5,9.0,0,127,0,10409,598,2095,2751,98,13,5045,226,1758,968
4,N10_00011,11,N10,N10-5,11.0,0,144,30,13011,560,1997,2699,80,25,5082,252,1756,1334


In [4]:
data_df.shape

(158909, 18)

# Centred log-ratio transformation

In [4]:
from scipy.stats.mstats import gmean

x = data_df.iloc[:, 6:].values
# replace the zero to 1
x = np.where(x == 0, 1, x) 
x = np.log(x / gmean(x, axis = 1).reshape(x.shape[0], 1))

print(x.shape)

(158909, 12)


In [11]:
norm_df = pd.concat(
    [pd.DataFrame(x, columns = data_df.columns[6:]), data_df[['composite_id', 'core_section', 'facies_merge_2']]],
    join = 'inner', axis = 1
)
#norm_df.set_index('composite_id', inplace=True)
norm_df.head()

,Ba,Br,Ca,Cl,Fe,K,Rb,S,Si,Sr,Ti,Zr,composite_id,core_section,facies_merge_2
0,-1.086255,-6.090201,2.662380,0.423029,1.686334,1.808210,-1.269920,-2.047150,2.526294,-1.027606,1.159724,1.255163,N10_00003,N10-5,0
1,-1.398651,-4.147523,2.654872,0.239180,1.455979,1.708623,-1.091166,-2.538085,2.320370,-0.903954,0.705483,0.994871,N10_00005,N10-5,0
2,-1.412732,-3.639810,2.660476,0.352536,1.500790,1.853252,-1.162507,-4.091795,2.520246,-0.704986,1.072991,1.051538,N10_00007,N10-5,0
3,-1.114063,-5.958250,3.292176,0.435341,1.689059,1.961470,-1.373283,-3.393301,2.567903,-0.537715,1.513682,0.916982,N10_00009,N10-5,0
4,-1.363775,-2.932391,3.139962,-0.005651,1.265813,1.567048,-1.951562,-3.114712,2.199872,-0.804159,1.137206,0.862349,N10_00011,N10-5,0


In [37]:
norm_df.set_index('composite_id').to_csv('data/XRF_ML_c.csv')

In [12]:
# make sure the order is sorted by the composite_id and then the order within section is sorted by the section depth simutaniously.
norm_df = norm_df.sort_values('composite_id')
norm_df.shape

(158909, 15)

# Build 2D data
Combine adjecent data points into each data point to be 2D array, like image. And then ravel it into 1D array in each data point. I adopt 17 data points to be adjecent data points, consistent to the rolling trick.

In [28]:
index = 8
len(norm_df.iloc[index-8: index+9, :-3].values.ravel()) == (17*12)

True

In [13]:
%%time
X = []
id_list = []
for section in norm_df.core_section.unique():
    for index in norm_df.index[norm_df.core_section == section][8:-8]:
        X.append(norm_df.iloc[index-8: index+9, :-3].values.ravel())
        id_list.append(index)
#ravel_df = pd.DataFrame(X, index=id_list)

CPU times: user 24.1 s, sys: 637 ms, total: 24.8 s
Wall time: 26.5 s


In [17]:
X_arr = np.array(X)
X_arr.shape

(153357, 204)

In [22]:
ravel_df = pd.DataFrame(X_arr, index=id_list)

In [ ]:
# This process can't be executed since the X is too big?
out_df = pd.concat([ravel_df.sort_index(), data_df.iloc[:, :6]], join='inner', axis=1).set_index('composite_id')
out_df

In [27]:
X_arr.nbytes/(1024**2)

238.68429565429688

# Export data
Since pd.DataFrame isn't feasible to use, we export the the data using np. Remember, this means there is no header and index in these csv files.

In [28]:
np.savetxt('data/XRF_ML_ravel_X.csv', X, delimiter=',')
np.savetxt('data/XRF_ML_ravel_y.csv', data_df.facies_merge_2[id_list], delimiter=',')
np.savetxt('data/XRF_ML_ravel_g.csv', data_df.core_section[id_list], delimiter=',', fmt='%s')
np.savetxt('data/XRF_ML_ravel_id.csv', data_df.composite_id[id_list], delimiter=',', fmt='%s')

In [18]:
x[:3].ravel().reshape(3, 12)

array([[-1.08625516, -6.09020147,  2.66238   ,  0.42302864,  1.68633356,
         1.80820962, -1.2699199 , -2.0471502 ,  2.52629392, -1.02760644,
         1.15972407,  1.25516337],
       [-1.39865066, -4.14752286,  2.65487191,  0.23918032,  1.45597877,
         1.70862299, -1.09116596, -2.53808495,  2.32036995, -0.90395442,
         0.70548349,  0.99487143],
       [-1.412732  , -3.63980954,  2.66047572,  0.35253586,  1.50079019,
         1.85325191, -1.16250749, -4.09179466,  2.52024637, -0.70498602,
         1.07299131,  1.05153834]])

In [16]:
x[:3]

array([[-1.08625516, -6.09020147,  2.66238   ,  0.42302864,  1.68633356,
         1.80820962, -1.2699199 , -2.0471502 ,  2.52629392, -1.02760644,
         1.15972407,  1.25516337],
       [-1.39865066, -4.14752286,  2.65487191,  0.23918032,  1.45597877,
         1.70862299, -1.09116596, -2.53808495,  2.32036995, -0.90395442,
         0.70548349,  0.99487143],
       [-1.412732  , -3.63980954,  2.66047572,  0.35253586,  1.50079019,
         1.85325191, -1.16250749, -4.09179466,  2.52024637, -0.70498602,
         1.07299131,  1.05153834]])